In [54]:
import numpy as np
import pandas as pd

data = pd.read_csv("data.csv", dtype = {"CRS_DEP_TIME":str, "DEP_TIME":str, "CRS_ARR_TIME":str, "ARR_TIME":str, "CANCELLED":bool, "DIVERTED":bool}, parse_dates=["FL_DATE"])

In [55]:
print(data.head())

   YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK    FL_DATE CARRIER ORIGIN  \
0  2015        1      1            23            5 2015-01-23      US    BUF   
1  2015        1      1            23            5 2015-01-23      US    CLT   
2  2015        1      1            23            5 2015-01-23      US    CLT   
3  2015        1      1            23            5 2015-01-23      US    LGA   
4  2015        1      1            23            5 2015-01-23      US    CLT   

  DEST CRS_DEP_TIME         ...          ARR_DELAY  ARR_DELAY_GROUP  \
0  CLT         0615         ...               -3.0             -1.0   
1  ALB         2220         ...               -7.0             -1.0   
2  BUF         0925         ...               -9.0             -1.0   
3  CLT         0630         ...              -19.0             -2.0   
4  BUF         2215         ...                7.0              0.0   

   CANCELLED DIVERTED DISTANCE  CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  \
0      False    F

Features = ['Month','Carrier','origin','Dest','distance'] <br>
y = ['dep_delay_group']

In [56]:
NYC_AIRPORTS = ["JFK", "LGA", "EWR"]
MIA_AIRPORTS = ["MCO", "FLL"]

data = data[(data["DEST"].isin(NYC_AIRPORTS) | data["ORIGIN"].isin(NYC_AIRPORTS))]
data = data[(data["DEST"].isin(MIA_AIRPORTS) | data["ORIGIN"].isin(MIA_AIRPORTS))]
data = data[data['CRS_DEP_TIME'].notna()]
data = data[data['CRS_ARR_TIME'].notna()]
data = data[data['ARR_TIME'].notna()]
data = data[data['DEP_TIME'].notna()]
#,'DEP_TIME','CRS_ARR_TIME','ARR_TIME'].notna()]]
#data.dropna(axis=0, how='any')
data

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,CARRIER,ORIGIN,DEST,CRS_DEP_TIME,...,ARR_DELAY,ARR_DELAY_GROUP,CANCELLED,DIVERTED,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
781,2015,1,1,20,2,2015-01-20,VX,FLL,JFK,0915,...,-22.0,-2.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
782,2015,1,1,20,2,2015-01-20,VX,JFK,FLL,0800,...,-2.0,-1.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
813,2015,1,1,21,3,2015-01-21,VX,FLL,JFK,0915,...,-8.0,-1.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
814,2015,1,1,21,3,2015-01-21,VX,JFK,FLL,0800,...,1.0,0.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
930,2015,1,1,15,4,2015-01-15,VX,FLL,JFK,0915,...,6.0,0.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
931,2015,1,1,15,4,2015-01-15,VX,JFK,FLL,0800,...,154.0,10.0,False,False,1069.0,154.0,0.0,0.0,0.0,0.0
960,2015,1,1,16,5,2015-01-16,VX,FLL,JFK,0915,...,7.0,0.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
961,2015,1,1,16,5,2015-01-16,VX,JFK,FLL,0800,...,-2.0,-1.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
983,2015,1,1,17,6,2015-01-17,VX,FLL,JFK,0915,...,-4.0,-1.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
984,2015,1,1,17,6,2015-01-17,VX,JFK,FLL,0800,...,-16.0,-2.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN


In [57]:
print (data[:5])
data.loc[data.CRS_DEP_TIME =='2400','CRS_DEP_TIME'] = '0000'
data.loc[data.DEP_TIME =='2400','DEP_TIME'] = '0000'
data.loc[data.CRS_ARR_TIME =='2400','CRS_ARR_TIME'] = '0000'
data.loc[data.ARR_TIME =='2400','ARR_TIME'] = '0000'
data.head()

     YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK    FL_DATE CARRIER  \
781  2015        1      1            20            2 2015-01-20      VX   
782  2015        1      1            20            2 2015-01-20      VX   
813  2015        1      1            21            3 2015-01-21      VX   
814  2015        1      1            21            3 2015-01-21      VX   
930  2015        1      1            15            4 2015-01-15      VX   

    ORIGIN DEST CRS_DEP_TIME         ...          ARR_DELAY  ARR_DELAY_GROUP  \
781    FLL  JFK         0915         ...              -22.0             -2.0   
782    JFK  FLL         0800         ...               -2.0             -1.0   
813    FLL  JFK         0915         ...               -8.0             -1.0   
814    JFK  FLL         0800         ...                1.0              0.0   
930    FLL  JFK         0915         ...                6.0              0.0   

     CANCELLED DIVERTED DISTANCE  CARRIER_DELAY  WEATHER_DELAY  NAS_

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,CARRIER,ORIGIN,DEST,CRS_DEP_TIME,...,ARR_DELAY,ARR_DELAY_GROUP,CANCELLED,DIVERTED,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
781,2015,1,1,20,2,2015-01-20,VX,FLL,JFK,0915,...,-22.0,-2.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
782,2015,1,1,20,2,2015-01-20,VX,JFK,FLL,0800,...,-2.0,-1.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
813,2015,1,1,21,3,2015-01-21,VX,FLL,JFK,0915,...,-8.0,-1.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
814,2015,1,1,21,3,2015-01-21,VX,JFK,FLL,0800,...,1.0,0.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN
930,2015,1,1,15,4,2015-01-15,VX,FLL,JFK,0915,...,6.0,0.0,False,False,1069.0,NaN,NaN,NaN,NaN,NaN


In [58]:
#data.head()
from datetime import datetime,timedelta
war_start = '2011-01-03 12:00'
datetime.strptime(war_start, '%Y-%m-%d %H:%M')


def to_time(fl_date,time_string):
    year_time  = fl_date.strftime('%Y-%m-%d')
    hour_time = ' '+time_string[:2]+':'+time_string[2:]
    time = year_time+hour_time
    return datetime.strptime(time, '%Y-%m-%d %H:%M')


In [59]:
data['CRS_DEP_TIME_DATETIME'] = data.apply(lambda row: to_time(row['FL_DATE'], row['CRS_DEP_TIME']), axis=1)
#print (data.ix[197660])
data['DEP_TIME_DATETIME'] = data.apply(lambda row: to_time(row['FL_DATE'], row['DEP_TIME']), axis=1)
#print (data.ix[1584706])
data['CRS_ARR_TIME_DATETIME'] = data.apply(lambda row: to_time(row['FL_DATE'], row['CRS_ARR_TIME']), axis=1)
data['ARR_TIME_DATETIME'] = data.apply(lambda row: to_time(row['FL_DATE'], row['ARR_TIME']), axis=1)

In [60]:
data['ARR_TIME_DATETIME'] = np.where(data['ARR_TIME_DATETIME'] < data['DEP_TIME_DATETIME'],data['ARR_TIME_DATETIME']+timedelta(days=1) ,data['ARR_TIME_DATETIME'])
data['CRS_ARR_TIME_DATETIME'] = np.where(data['CRS_ARR_TIME_DATETIME'] < data['CRS_DEP_TIME_DATETIME'],data['CRS_ARR_TIME_DATETIME']+timedelta(days=1) ,data['CRS_ARR_TIME_DATETIME'])

In [46]:
from sklearn import preprocessing
#['Month','Carrier','origin','Dest','distance'] 
total_table = data[['MONTH','CARRIER','ORIGIN','DEST','DISTANCE','DEP_DELAY_GROUP']]
#y_table = data['DEP_DELAY_GROUP'].dropna(axis=0, how='any')
total_table = total_table.dropna(axis=0, how='any')
total_table = total_table[total_table.DEP_DELAY_GROUP >0]
y_table = total_table['DEP_DELAY_GROUP']
y_table = y_table.reset_index(drop=True)
feature_table = total_table.drop(['DEP_DELAY_GROUP'],axis=1)
feature_table = feature_table.reset_index(drop=True)


feature_table['MONTH']= feature_table['MONTH'].astype('category')
feature_table = pd.get_dummies(feature_table)
feature_table.head()
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(feature_table[['DISTANCE']].values.astype(float))
#feature_table['DISTANCE']= 
feature_table['DISTANCE']= pd.DataFrame(x_scaled)
feature_table.head()


,DISTANCE,MONTH_1,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,...,ORIGIN_EWR,ORIGIN_FLL,ORIGIN_JFK,ORIGIN_LGA,ORIGIN_MCO,DEST_EWR,DEST_FLL,DEST_JFK,DEST_LGA,DEST_MCO
0,0.94964,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0.94964,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,0.94964,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,0.94964,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0.94964,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [77]:
#y_table = data['DEP_DELAY_GROUP']
#y_table = pd.get_dummies(y_table)
y_table.head()
#print (y_table.shape)

0    11.0
1     6.0
2     1.0
3     6.0
4     1.0
Name: DEP_DELAY_GROUP, dtype: float64

In [78]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)
print (feature_table.values)

clf.fit(feature_table.values, y_table.values)

[[ 0.94964029  1.          0.         ...,  0.          0.          0.        ]
 [ 0.94964029  1.          0.         ...,  1.          0.          0.        ]
 [ 0.94964029  1.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          1.         ...,  0.          0.          0.        ]
 [ 0.92086331  0.          1.         ...,  0.          0.          0.        ]
 [ 0.92086331  0.          1.         ...,  0.          0.          0.        ]]


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

In [83]:
prediction = clf.predict(feature_table.values)
count_predict =0
for i in prediction:
    if i != 1:
        print (i)
        count_predict+=1

count = 0
for i in y_table.values:
    if i !=1:
        count+=1
        #print (i)
print ('count = ',count)
print ("count_predict =",count_predict)
print (y_table.shape)
print (prediction)

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
count =  35139
count_predict = 24
(49910,)
[ 1.  1.  1. ...,  1.  1.  1.]


In [80]:
import requests
from bs4 import BeautifulSoup